In [ ]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urlparse

def google_search(keywords, sites):
    """
    Esegue una ricerca su Google per ciascun sito in base alle parole chiave specificate.

    Args:
      keywords: Una lista di parole chiave.
      sites: Una lista di siti su cui effettuare la ricerca.

    Returns:
      Un dizionario contenente i risultati della ricerca per ogni sito.
    """

    results = {}

    for site in sites:
        # Crea una richiesta HTTP a Google.
        url = f"https://www.google.com/search?q=site:{site} {'+'.join(keywords)}"
        response = requests.get(url)

        # Verifica se la richiesta ha avuto successo.
        if response.status_code == 200:
            # Estrai il codice HTML della risposta.
            html = response.text

            # Crea un oggetto BeautifulSoup per il codice HTML.
            soup = BeautifulSoup(html, "html.parser")

            # Estrai i link dai risultati della ricerca.
            links = [link.get('href') for link in soup.find_all("a", href=True) if "url?q=" in link['href']]

            # Filtra i link pertinenti e funzionanti.
            cleaned_links = []
            for link in links:
                cleaned_link = link.split("url?q=")[1].split("&sa=")[0]
                if site in cleaned_link and is_valid_url(cleaned_link):
                    cleaned_links.append(cleaned_link)

            results[site] = cleaned_links

        else:
            print(f"Errore durante la richiesta a Google per il sito {site}.")

    return results

def is_valid_url(url):
    """
    Verifica se un URL è valido.

    Args:
      url: URL da verificare.

    Returns:
      True se l'URL è valido, False altrimenti.
    """
    try:
        response = requests.head(url, allow_redirects=True)
        return response.status_code == 200
    except requests.RequestException:
        return False

def analyze_site_content(url, keywords):
    """
    Analizza il contenuto di una pagina web alla ricerca delle keywords specificate.

    Args:
      url: URL della pagina web da analizzare.
      keywords: Una lista di keywords da cercare nel contenuto della pagina.

    Returns:
      True se tutte le keywords sono trovate nel contenuto della pagina, False altrimenti.
    """
    try:
        response = requests.get(url)
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, "html.parser")
            text = soup.get_text().lower()
            for keyword in keywords:
                if keyword.lower() not in text:
                    return False
            return True
        return False
    except requests.RequestException:
        return False

if __name__ == "__main__":
    # Interfaccia utente per inserire i siti web e le parole chiave.
    sites = input("Inserisci i siti web su cui effettuare la ricerca (separati da virgola): ").split(",")
    keywords = input("Inserisci le parole chiave (separate da virgola): ").split(",")

    # Esegue la ricerca per ogni sito.
    results = google_search(keywords, sites)

    if results:
        # Analizza il contenuto dei siti trovati per le keywords specificate.
        for site, links in results.items():
            print(f"Risultati per {site}:")
            for link in links:
                if analyze_site_content(link, keywords):
                    print(f"{link}")
    else:
        print("Nessun risultato trovato.")
